In [6]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import pickle

from langchain_experimental.agents.agent_toolkits.csv.base import create_csv_agent

from openai import OpenAI

import time
import shutil
import os

In [7]:
os.environ["OPENAI_API_KEY"] = 


In [48]:
def LLMtext(path, prompt,model, temperature):
    df = pd.read_csv(path)
    cgm_values =",".join(str(round(x)) for x in df['CGM'].values)
    time_values =",".join(str(x) for x in df['Time'].values)
    gpt_user_prompt = prompt +f" CGM values: {cgm_values}, Time  values: {time_values}"
    client = OpenAI()
    gpt_assistant_prompt = "Estimate the answer from the text."

    message=[{"role": "system", "content": gpt_assistant_prompt}, {"role": "user", "content": gpt_user_prompt}]

    frequency_penalty=0.0

    response = client.chat.completions.create(
        model=model,
        messages = message,
        temperature=temperature,
    )
    return response.choices[0].message.content

In [34]:
def LLMcode(new_path, prompt,model, temperature,i=0,k=0):

    ###############
    ## Code adapted from
    ### L. Cheng, X. Li and L. Bing, Is GPT-4 a good data analyst?, arXiv [cs.CL] (May 2023).
    ###############
    df = pd.read_csv(new_path)
    small = df[['Time','CGM']]
    new_path = 'temp_files/_temp.txt'
    small.to_csv(new_path, sep='\t', index=False)
    small.to_csv(new_path)
    def get_gpt_result(system_role, question, max_tokens, temperature):
        client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])
        response = client.chat.completions.create(
            model=model,
            max_tokens=max_tokens,
            temperature=temperature,
            messages=[
                {"role": "system", "content": system_role},
                {"role": "user", "content": question}
            ]
        )
        return response 
    
    max_tokens = 2000
    system_role = '''Write code in Python to answer the question where the glucose column is "CGM" and the time column is "Time". All necessary packages are already installed.  
    Please save the answer in a file "answer.txt". '''
    question ="Question: " + prompt +' df = pd.read_csv('+new_path+')'
 
    response = get_gpt_result(system_role, question, max_tokens, float(temperature))
    text = response.choices[0].message.content
    print(text)
    try:
        string =text
        substring = "```"
        indexes = []
        index = string.find(substring)
        while index != -1:
            indexes.append(index)
            index = string.find(substring, index + 1)
        python = text[indexes[0] + 10:indexes[1]]
    except:
        python = text

    file = open(f"analyzecgm.py", "w")
    file.write(python)
    file.close()

    os.system(f"python analyzecgm.py")
    data = open(f"answer.txt", 'r').read()
    
    question = "Question: " + prompt + '\nData: \n' + data
    system_role = 'Generate response.'

    response = get_gpt_result(system_role, question, max_tokens, temperature)
    text = response.choices[0].message.content

    return text

In [35]:
def LLMcodechain(path, prompt,model, temperature):
    df = pd.read_csv(path)
    small = df[['Time','CGM']]
    new_path = 'temp_files/_temp.txt'
    small.to_csv(new_path, sep='\t', index=False)
    small.to_csv(new_path)
    agent = create_csv_agent(
        ChatOpenAI(temperature=temperature, model=model),
        new_path,
        verbose=True,return_intermediate_steps=True
        )
    response_= agent(prompt)
    response = response_['output']
    steps = response_['intermediate_steps']
    return response, steps

## Run

In [36]:
Questions = ['Q1:What was my mean glucose?',
'Q2:What was my maximum glucose?',
'Q3:What was the standard deviation of my glucose? ',
'Q4:What was my minimum glucose?',
'Q5:What was my percent time in range? ',
'Q6:What was my percent time in hyperglycemia?',
'Q7:What was my percent time in hypoglycemia?',
'Q8:What was my glycemic variability?',
'Q9:What was my percent time in severe hyperglycemia?',
'Q10:What is my estimated A1C?',
'Q11:What was my percent time in severe hypoglycemia?',
             
'Q12:What time was my blood glucose highest?',
'Q13:What day was my glucose control the most out of range?',
'Q14:What time of the day was my blood glucose lowest? ',
'Q15:When did my most recent episode of hypoglycemia start?',
'Q16:How long was my last episode of hypoglycemia?',
'Q17:What was my longest time spent in hyperglycemia? ',
'Q18:How many times did I experience hypoglycemia? ',
'Q19:What was my mean overnight blood glucose? ',
'Q20:What period of the day did I have the highest blood glucose?',
'Q21:Did I have noctural hypoglycemia? ',
'Q22:What was my highest glucose reading during dinner? ',

'Q23:What percent of time was my CGM active?',
'Q24:How many times did my sensor disconnect?',
'Q25:Was my low blood glucose likely due to sensor error?',
'Q26:Are there any artifacts in the CGM data?',
                  
'Q27:Was my average glucose control today better than yesterday?',
'Q28:Was my time in range improved this week compared to last week? ',
'Q29:Was my max glucose lower today than yesterday?',
'Q30:Did I spend less time in hypoglycemia this week than last week?']

In [1]:
model = "gpt-4-0125-preview"
temperature= .1

## Iterate through Cases

In [ ]:
all_results = {}
k=0
for prompt in Questions:
    prompt = prompt[4:]
    print('\n\n ---')
    time.sleep(2)
    print(prompt)
    print('\n\n')
    k=k+1
    all_results[f'Q{k}'] = {}
    all_results['Question']=prompt


    new_path = f'data.csv'
    all_results[f'Q{k}']={}
    savepath=f'files/dat_{k}'
    if not os.path.exists(savepath):
        os.makedirs(savepath)

    try:
        print('\nCode:')
        response = LLMcode(new_path, prompt,model, temperature,i,k)
        print(response)
        all_results[f'Q{k}']['LLM-code'] = response
        shutil.move('answer.txt',savepath+'/out.txt') 
        shutil.move('analyzecgm.py',savepath+'/run.py') 
    except Exception as e: 
        print(e)
        print(f'{k}  failed')      
    try:
        print('\nCodeChain:')
        response,steps =LLMcodechain(new_path, prompt,model, temperature)
        print(response)
        all_results[f'Q{k}']['LLM-codechain'] = response

    except Exception as e: 
        print(e)
        print(f'{k} failed')